# 🍎 AIProjectClient에서 Phi-4 모델 사용 🍏

**Phi-4**는 적은 비용으로 GPT-4o에 가까운 기능을 제공하는 것을 목표로 하는 차세대 개방형 모델로, 많은 기업 또는 개인 사용 사례에 이상적입니다. 특히 연쇄 추론과 RAG(검색 증강 세대) 시나리오에 적합합니다.

이 노트북은 다음과 같이 진행됩니다:
1. Azure AI 파운드리 환경에 맞게 `AIProjectClient`를 **초기화**합니다.
2. `azure-ai-inference`를 사용하여 **Phi-4** 모델과 **채팅**합니다.
3. 고지 사항 및 건강 Q&A가 포함된 건강 및 피트니스 예제를 **표시**합니다.
4. 강력한 추론 기능을 갖춘 GPT-4에 대한 저렴한 대안을 이용할 수 있습니다. 🏋️

> **면책조항**: 이것은 의학적 조언이 아닙니다. 전문가와 상담하시기 바랍니다..

## 왜 Phi-4인가?
Phi-4는 높은 추론 성능을 위해 선별된 데이터로 학습된 14B 파라미터 모델입니다.
- **비용 효율적**: GPT-4 수준의 가격보다 저렴하게 많은 작업에서 GPT-4 수준의 성능을 얻을 수 있습니다.
- **추론 및 RAG**: 연쇄 추론 단계 및 검색 증강 생성 워크플로우에 적합합니다.
- **충분한 컨텍스트 윈도우**: 16K 토큰으로 더 많은 컨텍스트 또는 더 긴 사용자 대화가 가능합니다.

<img src="./seq-diagrams/4-phi-4.png" width="30%"/>


## 1. 설정

아래에서 필요한 라이브러리를 설치하고 가져오겠습니다:
- **azure-ai-projects**: `AIProjectClient`용
- **azure-ai-inference**: 모델, 특히 채팅 완료를 호출하기 위한 라이브러리.
- **azure-identity**: `DefaultAzureCredential`용

`.env` 파일에 다음을 확인하세요:
```bash
PROJECT_CONNECTION_STRING=<your-conn-string>
SERVERLESS_MODEL_NAME=phi-4
```

> **주**: 여기서 도움이 될 중요한 개념을 다루고 있으므로 [`3-basic-rag.ipynb`](./3-basic-rag.ipynb) 노트북을 완수하고 이 노트북을 수행하세요.

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage

from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

conn_string = os.getenv("PROJECT_CONNECTION_STRING")
phi4_deployment = os.getenv("SERVERLESS_MODEL_NAME", "phi-4")

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=conn_string,
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)

## 2. Phi-4와 채팅하기 🍏
건강 및 피트니스 맥락에서 **Phi-4**를 사용한 간단한 대화를 시연해 보겠습니다. 어시스턴트의 역할을 명확히 하는 시스템 프롬프트를 정의하겠습니다. 그런 다음 몇 가지 사용자 질문을 해보겠습니다.

> Phi-4는 연쇄 추론에 매우 적합하다는 것을 알 수 있습니다. 추론 단계를 설명해 보겠습니다.


In [ ]:
def chat_with_phi4(user_question, chain_of_thought=False):
    """Send a chat request to the Phi-4 model with optional chain-of-thought."""
    # We'll define a system message with disclaimers:
    system_prompt = (
        "You are a Phi-4 AI assistant, focusing on health and fitness.\n"
        "Remind users that you are not a medical professional, but can provide general info.\n"
    )

    # We can optionally instruct the model to show chain-of-thought. (Use carefully in production.)
    if chain_of_thought:
        system_prompt += "Please show your step-by-step reasoning in your answer.\n"

    # We create messages for system + user.
    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.8,  # a bit creative
            top_p=0.9,
            max_tokens=400,
        )

    return response.choices[0].message.content

# Example usage:
question = "I'm training for a 5K. Any tips on a weekly workout schedule?"
answer = chat_with_phi4(question, chain_of_thought=True)
print("🗣️ User:", question)
print("🤖 Phi-4:", answer)

## 3. RAG와 유사한 예제(스텁)
Phi-4는 외부 컨텍스트를 제공하고 모델이 이를 추론하도록 하는 검색 증강 생성 시나리오에서도 탁월한 성능을 발휘합니다. 다음은 검색된 텍스트를 컨텍스트로 전달하는 방법을 보여주는 **스텁** 예제입니다.

> 실제 시나리오에서는 관련 구절을 임베드하고 검색한 다음 사용자/시스템 메시지에 전달합니다.


In [ ]:
def chat_with_phi4_rag(user_question, retrieved_doc):
    """Simulate an RAG flow by appending retrieved context to the system prompt."""
    system_prompt = (
        "You are Phi-4, helpful fitness AI.\n"
        "We have some context from the user's knowledge base: \n"
        f"{retrieved_doc}\n"
        "Please use this context to help your answer. If the context doesn't help, say so.\n"
    )

    system_msg = SystemMessage(content=system_prompt)
    user_msg = UserMessage(content=user_question)

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            model=phi4_deployment,
            messages=[system_msg, user_msg],
            temperature=0.3,
            max_tokens=300,
        )
    return response.choices[0].message.content

# Let's define a dummy doc snippet:
doc_snippet = "Recommended to run 3 times per week and mix with cross-training.\n" \
              "Include rest days or active recovery days for muscle repair."

user_q = "How often should I run weekly to prepare for a 5K?"
rag_answer = chat_with_phi4_rag(user_q, doc_snippet)
print("🗣️ User:", user_q)
print("🤖 Phi-4 (RAG):", rag_answer)

## 4. 마무리 및 모범 사례
1. **Chain-of-Thought**: 디버깅이나 특정 QA 작업에 유용하지만 최종 사용자에게 Chain-of-Thought를 공개하는 것에 유의하세요.r
2. **RAG**: 검색 결과와 함께 `azure-ai-inference`를 사용하여 답변의 근거를 마련하세요.
3. **OpenTelemetry**: 완전한 통합 가시성을 위해 선택적으로 `opentelemetry-sdk` 및 `azure-core-tracing-opentelemetry`를 통합합니다.
4. **Evaluate**: 모델 성능을 측정하려면 `azure-ai-evaluation`을 사용합니다.
5. **Cost & Performance**: Phi-4는 저렴한 비용으로 GPT-4에 가까운 성능을 제공하는 것을 목표로 합니다. 도메인 요구 사항에 맞게 평가하세요.

## 🎉 축하합니다!
방법을 확인하셨습니다:
- `AIProjectClient` 및 `azure-ai-inference`를 **Phi-4**와 함께 사용하세요.
- chain-of-thought을 사용하여 **chat** 흐름을 만듭니다.
- RAG** 시나리오를 stub.

> Happy hacking! 🏋️
